<h1 style="color:#8BB065;font-size:50px;"><strong>My first <strong style="color:#974949"> Convolutional Neural Network</strong></strong></h1>


<img src="https://i.ibb.co/VMSgnJ5/header.png" alt="header" border="0">

Learning CNN from this notebook: [Link](https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1)

In [ ]:
#load in packages
import os
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc
from tqdm import tqdm

#images
import cv2

#modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#source path（Pawpularity 比赛数据所在的位置）
path = '../input/petfinder-pawpularity-score/'

#获取元数据（.csv 数据）并将其放入 DataFrames
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

#获取图像数据（.jpg 数据）并将其放入文件名列表中
train_jpg = glob(path + "train/*.jpg")
test_jpg = glob(path + "test/*.jpg")

In [ ]:
train_df.shape, len(train_jpg)#显示数据列表大小

In [ ]:
train_df.head(5)#显示前5行

<h1 style="color:#189AB4;font-size:20px;"><strong>EDA<strong style="color:black"></strong></strong></h1>

In [ ]:
sns.histplot(train_df['Pawpularity']);#绘制'Pawpularity'的直方图

plt.axvline(train_df['Pawpularity'].mean(),color='red')
plt.axvline(train_df['Pawpularity'].median(),color='green')

In [ ]:
train_df[['Pawpularity']].describe().T

In [ ]:
#修改 Id，使每个 Id 都是完整的图像路径。 
def train_id_to_path(x):
    return '../input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '../input/petfinder-pawpularity-score/test/' + x + ".jpg"


In [ ]:
#读入数据并删除不必要的列
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
print(train)
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)
print(train)
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

In [ ]:
#将 .jpg 扩展名添加到图像文件名 ID
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
#binning columns to test models 将列分档以测试模型
train['two_bin_pawp'] = pd.qcut(train['Pawpularity'], q=2, labels=False)
train = train.astype({"two_bin_pawp": str})

train['four_bin_pawp'] = pd.qcut(train['Pawpularity'], q=4, labels=False)
train = train.astype({"four_bin_pawp": str})

train['ten_bin_pawp'] = pd.qcut(train['Pawpularity'], q=10, labels=False)
train = train.astype({"ten_bin_pawp": str})

In [ ]:
#设置你想要的图像尺寸
image_height = 128
image_width = 128

#define a function that accepts an image url and outputs an eager tensor 定义一个函数，它接受一个图像并输出一个张量
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    #image = tf.image.resize_with_pad(image, image_height, image_width) #可选填充以保留原始尺寸
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
#获取train文件夹中的所有图像并将它们的张量放在一个列表中
X = []
for img in tqdm(train['img_path']):
    new_img_tensor = path_to_eagertensor(img)
    X.append(new_img_tensor)
    
print(type(X),len(X))
X = np.array(X)
print(type(X),X.shape)

In [ ]:
#获取 test 文件夹中的所有图像并将它们的张量放在一个列表中
X_submission = []
for img in tqdm(test['img_path']):
    new_img_tensor = path_to_eagertensor(img)
    X_submission.append(new_img_tensor)
    
print(type(X_submission),len(X_submission))
X_submission = np.array(X_submission)
print(type(X_submission),X_submission.shape)

In [ ]:
# 获取目标变量Pawpularity
y = train['Pawpularity']
print(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7) #划分训练集和测试集为9：1

In [ ]:
print(type(x_train),x_train.shape)

In [ ]:
#显示每个新数组的形状
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
Z=train_df.drop(['Id', "Pawpularity"], axis=1)
Z=np.array(Z)
#print(type(Z),Z.shape)
z_train, z_test = train_test_split(Z, test_size=0.1, random_state=7) #划分元数据9：1
print(type(z_train),z_train.shape)

In [ ]:
# def get_model():
#     # image input model
#     in_put = tf.keras.layers.Input(shape=(X.shape[1],X.shape[2],3),name='in_put')
#     x = keras.layers.Conv2D(filters = 16, kernel_size=[5,5], activation = 'relu')(in_put)#二维卷积
#     x = keras.layers.BatchNormalization()(x)#归一化
#     ####
#     x = keras.layers.Conv2D(filters = 32, kernel_size=[5,5], activation = 'relu')(x)#激活函数
#     x = keras.layers.BatchNormalization()(x)#归一化
#     x = keras.layers.Dropout(0.25)(x)#丢弃，防止过拟合
#     ####
#     x = keras.layers.Conv2D(filters = 64, kernel_size=[3,3],  activation='relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Dropout(0.25)(x)
#     ####
#     x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),  activation = 'relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x) #最大池化层，减小数据敏感程度
#     ####
#     x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),activation = 'relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Dropout(0.25)(x)
#     x = tf.keras.layers.Flatten()(x) #扁平层，将二维数据变为以一维，使之输出到全连接层上
#     ####
#     out_put = layers.Dense(512, activation='relu')(x)
#     in_model = tf.keras.Model(in_put, out_put)
 
#     # other data Model
#     meta_input = layers.Input(shape=Z.shape[1], name='meta_input')
#     x = layers.Dense(64, activation='relu')(meta_input)
#     x = layers.Dropout(0.25)(x)
#     meta_output = layers.Dense(32, activation='relu')(x)
#     meta_model = tf.keras.Model(meta_input, meta_output)

#     # concatinating Model layers
#     con_layer = layers.Concatenate(name = 'con_layer')([in_model.output, meta_model.output])
#     combined_dropout = layers.Dropout(0.5)(con_layer)
#     combined_dence = layers.Dense(128, activation='relu')(combined_dropout)
#     final_dropout = layers.Dropout(0.5)(combined_dence)
#     output_layer = layers.Dense(1, activation='relu')(final_dropout)
#     model = tf.keras.Model(inputs = [in_model.input, meta_model.input], outputs=output_layer)
    
#     return model

In [ ]:
def get_model(): #定义模型
   
    in_put = keras.Input(shape=(X.shape[1],X.shape[2],3))

    x = keras.layers.Conv2D(filters = 16, kernel_size=[5,5], activation = 'sigmoid')(in_put)#二维卷积
   
    x = keras.layers.BatchNormalization()(x)#归一化
    ####

    x = keras.layers.Conv2D(filters = 32, kernel_size=[5,5], activation = 'sigmoid')(x)#激活函数

    x = keras.layers.BatchNormalization()(x)#归一化

    x = keras.layers.Dropout(0.25)(x)#丢弃，防止过拟合
    ####

    x = keras.layers.Conv2D(filters = 64, kernel_size=[3,3],  activation='sigmoid')(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dropout(0.25)(x)
    ####

    x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),  activation = 'sigmoid')(x)

    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x) #最大池化层，减小数据敏感程度
    ####

    x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),activation = 'sigmoid')(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Flatten()(x) #扁平层，将二维数据变为以一维，使之输出到全连接层上
    ####

    x = tf.keras.layers.Dense(512, activation = "sigmoid")(x)#全连接层，该层的效果是将输入的最高维(也就是最后一维)转成指定的维度数
    
    x = tf.keras.layers.Dense(128, activation = "sigmoid")(x)
    
    x = tf.keras.layers.Dense(64, activation = "sigmoid")(x)

    x = tf.keras.layers.Dropout(0.5)(x)

    out_put = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs = in_put, outputs = out_put)
    
    return model

In [ ]:
model = get_model()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

In [ ]:
#编译模型 

model.compile(
    loss = 'mse', #损失函数为mse
    optimizer = 'Adam', #优化器为Adam
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse")]) #评估标准为rmse

In [ ]:
#数据增强，扩充数据集大小，增强模型的泛化能力。
data_augmentation = ImageDataGenerator(rotation_range=15, zoom_range=0.15, width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [ ]:
kall = keras.callbacks.EarlyStopping(monitor='val_rmse',patience=10,restore_best_weights=True)#提前停止，防止过拟合

history = model.fit(
#     x_train,
#     y_train,
#     batch_size=32,
    #data_augmentation.flow({'input_1': x_train, 'input_2': z_train}, {'output': y_train},batch_size=32), 
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train) // 32,
    epochs =600, callbacks=[kall]
)

In [ ]:
plt.figure()
plt.plot(history.history["rmse"], label="train_rmse")
plt.plot(history.history["val_rmse"], label="val_rmse")
plt.title("RMSE train/validation by Epoch")
plt.xlabel("Epoch #")
plt.ylabel("RMSE")
plt.legend(loc="upper right");

In [ ]:
#predict on the submission data 对提交数据进行预测
cnn_pred = model.predict(X_submission)
print(X_submission.shape, type(X_submission))
print(cnn_pred.shape, type(cnn_pred))

In [ ]:
#将提交预测与其关联的 ID 放在一起
cnn = pd.DataFrame()
cnn['Id'] = test['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [ ]:
testing_example_image = plt.imread('../input/petfinder-pawpularity-score/test/4128bae22183829d2b5fea10effdb0c3.jpg') 
print(testing_example_image.shape)
#then plt.imshow() can display it for you
plt.imshow(testing_example_image)
plt.title('First Testing Image \n Predicted Pawpularity = {}'.format(cnn['Pawpularity'].iloc[0])) 
plt.axis('off') #turns off the gridlines
plt.show()